<a href="https://colab.research.google.com/github/HdSanch/RI_2024B/blob/main/Examen1BIM_SH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen Bimestral – Diseño de un Sistema Básico de Recuperación de Información

### Nombre: Hernán Sánchez

# Librerias usadas:

1. **Pandas**: Usa para manipulación de DataFrames y Series, estructuras de datos que pandas ofrece.

2. **Numpy**: Se usa para realizar cálculos numéricos, crear matrices o manejar valores numéricos dentro del análisis.

3. **sklearn.feature_extraction.text**: Se utiliza para convertir texto en una representación numérica, mediante TF-IDF.

4. **sklearn.metrics.pairwise**: Ayuda a calcular la similitud entre vectores utilizando la métrica del coseno.

5. **nltk**: Se usa para tareas como tokenización, lematización y análisis gramatical.

6. **nltk.corpus**: Ayuda a cargar una lista con stopwords.

7. **nltk.tokenize**: Se usa para tokenizar texto.

8. **nltk.stem**: Se utiliza para convertir palabras a su forma raíz.

9. **re**: Se utiliza para buscar, reemplazar, dividir o validar patrones de texto utilizando expresiones regulares.

n_results: Variable que establece cuantos resultados quieres visualizar en la búsqueda.

In [69]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import re

# Variables globales
movies_df = None
reviews_df = None
vectorizer = None
tfidf_matrix = None
tokenizer = None
stop_words = None
stemmer = None
n_results = 35

# Obtencion y preprocesamiento de la data

Primero se descarga la data de:
https://www.kaggle.com/datasets/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset

Aqui se descarga un zip con dos archivos los cuales pasarana a ser preprocesados, una vez descomprimidos pasamos a cargarlos.

- load_data(movies_path, reviews_path): Cargar los datos de películas y reseñas desde los archivos CSV y los convierte en dataframes.


In [80]:
def load_data(movies_path, reviews_path):
    """Cargar y preparar los datos de películas y reseñas"""
    global movies_df, reviews_df

    try:
        movies_df = pd.read_csv(movies_path)
        reviews_df = pd.read_csv(reviews_path)

        # Mostrar totales
        total_movies = len(movies_df)
        total_reviews = len(reviews_df)
        print("\n=== Estadísticas generales ===")
        print(f"Total de películas: {total_movies:,}")
        print(f"Total de reseñas: {total_reviews:,}")
        print(f"Promedio de reseñas por película: {total_reviews/total_movies:.2f}")

        # Mostrar primeras 5 filas de cada DataFrame
        print("\n=== Primeras 5 películas ===")
        print(movies_df.head())

        print("\n=== Primeras 5 reseñas ===")
        print(reviews_df.head())

        # Limpieza de datos
        text_columns = ['movie_title', 'movie_info', 'critics_consensus']
        for col in text_columns:
            if col in movies_df.columns:
                movies_df[col] = movies_df[col].fillna('')

        rating_columns = ['tomatometer_rating', 'audience_rating']
        for col in rating_columns:
            if col in movies_df.columns:
                movies_df[col] = movies_df[col].fillna('No disponible')

        # Combinar información para búsqueda (solo si existen las columnas)
        text_cols = [col for col in text_columns if col in movies_df.columns]
        if text_cols:
            movies_df['combined_text'] = movies_df[text_cols].fillna('').agg(' '.join, axis=1)

        print("\nDatos cargados exitosamente!")
        return True

    except FileNotFoundError:
        print("\nError: No se encontraron los archivos de datos.")
        print("Archivos necesarios:")
        print(f"- {movies_path}")
        print(f"- {reviews_path}")
        return False

    except Exception as e:
        print(f"\nError al cargar los datos: {str(e)}")
        import traceback
        print("\nDetalles del error:")
        print(traceback.format_exc())
        return False

1. initialize_nlp_components(): Inicializar los componentes necesarios para el procesamiento de texto, como el tokenizador, las palabras de parada y el stemmer.
2. clean_query(text): Convierte el texto a minúsculas.Elimina caracteres especiales o puntuación, reemplazándolos con espacios. Reduce múltiples espacios consecutivos a uno solo. Devuelve el texto limpio y sin espacios al inicio o al final.
3. preprocess_text(text): Limpia el texto utilizando clean_query.Tokeniza el texto con el tokenizer configurado y elimina stop words.Aplica el stemming a cada palabra. Combina las palabras procesadas en una sola cadena.


In [81]:
def initialize_nlp_components():
    """Inicializar componentes de procesamiento de lenguaje natural"""
    global tokenizer, stop_words, stemmer

    try:
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))
        stemmer = PorterStemmer()
        tokenizer = RegexpTokenizer(r'\w+')
        return True
    except Exception as e:
        print(f"Error al inicializar componentes NLP: {str(e)}")
        return False

def clean_query(text):
    """Limpieza básica de la query"""
    if not isinstance(text, str) or pd.isna(text):
        return ""

    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def preprocess_text(text):
    """Preprocesamiento completo del texto para búsqueda"""
    if pd.isna(text) or not isinstance(text, str):
        return ""

    # Limpieza básica
    text = clean_query(text)

    # Tokenización y procesamiento avanzado
    tokens = tokenizer.tokenize(text)
    tokens = [stemmer.stem(token) for token in tokens
             if token not in stop_words and len(token) > 2]

    return ' '.join(tokens)


# Vectorización
- build_search_index(): Usa un TfidfVectorizer para convertir el texto procesado en una matriz TF-IDF. Maneja errores durante el proceso de construcción.


In [82]:
def build_search_index():
    """Construir el índice de búsqueda usando TF-IDF"""
    global vectorizer, tfidf_matrix

    print("Construyendo índice de búsqueda...")
    try:
        # Preprocesar textos
        movies_df['processed_text'] = movies_df['combined_text'].apply(
            preprocess_text
        )

        # Crear y entrenar vectorizador TF-IDF
        vectorizer = TfidfVectorizer(max_features=5000)
        tfidf_matrix = vectorizer.fit_transform(
            movies_df['processed_text']
        )
        print("Índice de búsqueda construido exitosamente!")
        return True

    except Exception as e:
        print(f"Error al construir el índice: {str(e)}")
        return False

# Preprocesamiento, vectorización y búsqueda de la query

- search_movies(query): Preprocesa la consulta ingresada usando preprocess_text.Vectoriza la consulta utilizando el vectorizer entrenado.
Calcula la similitud coseno entre el vector de la consulta y la matriz TF-IDF.
Identifica las películas más relevantes, las ordena por relevancia y selecciona las principales según n_results.
Devuelve una lista de resultados con título, puntajes de relevancia, información, y calificaciones.

- format_movie_info(info_text, max_length=200): Formatear la información de una película para mostrarla.

- display_results(query, results) Mostrar los resultados de la búsqueda en la consola. Imprime el encabezado con la consulta.
Si no hay resultados, informa al usuario.

In [83]:
def search_movies(query):
    """Realizar búsqueda de películas"""
    global n_results

    try:
        # Preprocesar la consulta
        processed_query = preprocess_text(query)

        if not processed_query:
            return []

        # Vectorizar la consulta
        query_vector = vectorizer.transform([processed_query])

        # Calcular similitudes
        similarities = cosine_similarity(query_vector, tfidf_matrix)[0]

        # Filtrar y ordenar resultados
        relevant_indices = np.where(similarities > 0)[0]

        if len(relevant_indices) == 0:
            return []

        top_indices = relevant_indices[
            np.argsort(similarities[relevant_indices])[-n_results:][::-1]
        ]

        # Preparar resultados
        results = []
        for idx in top_indices:
            movie = movies_df.iloc[idx]
            result = {
                'title': movie['movie_title'],
                'score': similarities[idx],
                'info': movie['movie_info'],
                'tomatometer_rating': movie['tomatometer_rating'],
                'audience_rating': movie['audience_rating']
            }
            results.append(result)

        return results

    except Exception as e:
        print(f"Error en la búsqueda: {str(e)}")
        return []

def format_movie_info(info_text, max_length=200):
    """Formatear información de la película"""
    if pd.isna(info_text) or not isinstance(info_text, str):
        return "Información no disponible"

    if len(info_text) > max_length:
        return f"{info_text[:max_length]}..."
    return info_text

def display_results(query, results):
    """Mostrar resultados de la búsqueda"""
    print(f"\nResultados para: '{query}'")
    print("-" * 50)

    if not results:
        print("No se encontraron resultados.")
        return

    for i, result in enumerate(results, 1):
        print(f"\n{i}. {result['title']}")
        print(f"   Relevancia: {result['score']:.3f}")
        print(f"   Calificación críticos: {result['tomatometer_rating']}")
        print(f"   Calificación audiencia: {result['audience_rating']}")
        print(f"   Info: {format_movie_info(result['info'])}")


# Ejecución

- initialize_search(): Inicializa los componentes de NLP, carga los datos y construye el índice de búsqueda. Ofrece un menú interactivo para realizar búsquedas, cambiar el número de resultados (resultados N) o salir (salir/exit). Procesa las entradas, ejecuta búsquedas, y muestra los resultados en tiempo real.
- main(): Llama a initialize_search para iniciar el sistema.


In [86]:
sdef initialize_search():
    """Ejecutar el sistema en modo interactivo"""
    global n_results

    print("\nInicializando sistema de búsqueda de películas...")

    # Inicializar componentes
    if not initialize_nlp_components():
        return

    # Cargar datos
    if not load_data('rotten_tomatoes_movies.csv',
                     'rotten_tomatoes_critic_reviews.csv'):
        return

    # Construir índice
    if not build_search_index():
        return

    print("\n¡Sistema listo!")
    print("\nInstrucciones:")
    print("- Escribe tu búsqueda y presiona Enter")
    print("- Para salir: 'salir' o 'exit'")

    while True:
        print("\n" + "="*50)
        query = input("\nBúsqueda (o 'salir'): ").strip()

        # Verificar comando de salida
        if query.lower() in ['salir', 'exit']:
            print("\n¡Gracias por usar el sistema!")
            break

        # Verificar cambio de número de resultados
        if query.lower().startswith('resultados '):
            try:
                n_results = int(query.split()[1])
                print(f"\nNúmero de resultados cambiado a: {n_results}")
                continue
            except:
                print("Formato inválido. Uso: 'resultados N'")
                continue

        # Realizar búsqueda
        if query:
            results = search_movies(query)
            display_results(query, results)
        else:
            print("Por favor ingresa una búsqueda válida")

def main():
    """Función principal"""
    initialize_search()

if __name__ == "__main__":
    main()



Inicializando sistema de búsqueda de películas...

=== Estadísticas generales ===
Total de películas: 17,712
Total de reseñas: 1,130,017
Promedio de reseñas por película: 63.80

=== Primeras 5 películas ===
                    rotten_tomatoes_link                                         movie_title  \
0                              m/0814255  Percy Jackson & the Olympians: The Lightning Thief   
1                              m/0878835                                         Please Give   
2                                   m/10                                                  10   
3                 m/1000013-12_angry_men                     12 Angry Men (Twelve Angry Men)   
4  m/1000079-20000_leagues_under_the_sea                        20,000 Leagues Under The Sea   

                                                                                                                                                                                                                       